In [2]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import json
import ast
import matplotlib.image as mpimg
import numpy as np
import pathlib
from shutil import copyfile

from metaspace.sm_annotation_utils import SMInstance
import getpass

# this is the local copy of the method edited to run on its own.
# currently broken
from results_local import results as results2

from main_functions import *

from metaspace.sm_annotation_utils import GraphQLClient
del GraphQLClient.DEFAULT_ANNOTATION_FILTER['hasHiddenAdduct']

sm = SMInstance(host='https://beta.metaspace2020.eu')

import types
# Assign external function as method to object
sm.results2 = types.MethodType(results2, sm)

sm

SMInstance(https://beta.metaspace2020.eu/graphql)

-Goal: extract ion images from METASPACE and put in folders by assigned ID with parents and fragments together.

Previous notebook: 
http://localhost:8888/notebooks/PycharmProjects/word2vec/database_expt_msms_to_METASPACE.ipynb

Tutorial template:
https://github.com/metaspace2020/metaspace/blob/master/metaspace/python-client/example/iso_img_retrieval.ipynb


In [4]:
# Load secret stuff!
f = open('/Users/dis/.metaspace.json', "r")
secret = (f.read())
secret = secret.replace('\n','')
secret = ast.literal_eval(secret)
f.close()

# Logon
#api_key = getpass.getpass(prompt='API key: ', stream=None)
sm.login(secret['email'], secret['password'])

In [5]:
def metaspace_hotspot_removal(img):
  awkwardness = np.max(img) / 255 # METASPACE holdover from 8-bit
  hot_thresold = np.percentile(img[img >= awkwardness], 99) or 1
  return np.clip(img, 0, hot_thresold) / hot_thresold

In [64]:
cm3_wb_ds_prod_beta_dict = {'2016-09-21_16h07m45s': '2020-04-29_00h28m08s',
                            '2017-05-17_19h49m04s': '2020-04-29_00h28m11s',
                            '2017-05-17_19h50m07s': '2020-04-29_00h28m14s',
                            '2017-05-29_07h28m52s': '2020-04-29_00h28m17s',
                            '2017-08-03_15h09m06s': '2020-04-29_00h28m28s',
                            '2017-08-03_15h09m51s': '2020-04-29_00h28m31s',
                            '2017-08-11_07h59m58s': '2020-04-29_00h28m35s',
                            '2017-08-11_08h01m02s': '2020-04-29_00h28m39s'
                           }

In [79]:
# METASPACE reports manually downloaded re: API issues.

In [80]:
# Run when METASPACE reports change, loop to make msms reports.
for prod_id, beta_id in cm3_wb_ds_prod_beta_dict.items():
    print(beta_id)
    target = 'any_results/metaspace_report/' + beta_id + '.csv'
    fh = glob.glob(target)
    print(fh)
    ms_out = pd.read_csv(fh[0], header=2)
    print(ms_out.shape)
    ms_out2 = extract_results_METASPACE(ms_out)
    print(ms_out2.shape)
    ms_out2.to_csv('any_results/msms_report/' + prod_id + '_msms_report.csv')

2020-04-29_00h28m08s
['any_results/metaspace_report/2020-04-29_00h28m08s.csv']
(54, 20)
(806, 12)
2020-04-29_00h28m11s
['any_results/metaspace_report/2020-04-29_00h28m11s.csv']
(1165, 20)
(6879, 12)
2020-04-29_00h28m14s
['any_results/metaspace_report/2020-04-29_00h28m14s.csv']
(1266, 20)
(8203, 12)
2020-04-29_00h28m17s
['any_results/metaspace_report/2020-04-29_00h28m17s.csv']
(244, 20)
(2889, 12)
2020-04-29_00h28m28s
['any_results/metaspace_report/2020-04-29_00h28m28s.csv']
(113, 20)
(1714, 12)
2020-04-29_00h28m31s
['any_results/metaspace_report/2020-04-29_00h28m31s.csv']
(108, 20)
(1344, 12)
2020-04-29_00h28m35s
['any_results/metaspace_report/2020-04-29_00h28m35s.csv']
(137, 20)
(1576, 12)
2020-04-29_00h28m39s
['any_results/metaspace_report/2020-04-29_00h28m39s.csv']
(148, 20)
(1844, 12)


In [52]:
# For Theo manual msms report 1) has parent, and 2) 1+ fragment
msms_reps = glob.glob('any_results/msms_report/*.csv')
for rep in msms_reps:
    df = pd.read_csv(rep)
    df = df[(df.parent_y == 1) & (df.n_frag_y > 0)]
    ds_name = rep.split('/')[-1]
    ds_name = ds_name.split('.')[0]
    outpath = 'any_results/msms_theo_man_report/' + ds_name + '.csv'
    df.to_csv(outpath)

In [81]:
def dl_img(main_id, beta_id, db, fdr_max, save_img):
    # Load and save dataset as image or arrays (saved as df)
    ds = sm.dataset(id=beta_id)
    print(ds)
    
    # Generate and save images
    x = ds.all_annotation_images(fdr=fdr_max, 
                                 database=db,
                                 only_first_isotope=True,
                                 scale_intensity=False,
                                 hasHiddenAdduct=True)
    if x == []:
        return 'Error, empty annotations!'
    else:
        for n in x:
            if save_img == True:
                image = metaspace_hotspot_removal(n._images[0])
                plt.imshow(image)
                pathlib.Path('formula/' + main_id + '/').mkdir(parents=True, exist_ok=True)
                img_name = 'formula/' + main_id + '/' + n.formula + '.png'
                plt.imsave(img_name, image)
            else:
                # For saving arrays as df per Theo's request
                df = pd.DataFrame(data=metaspace_hotspot_removal(n._images[0]))
                pathlib.Path('any_results/formula_arr/' + main_id + '/').mkdir(parents=True, exist_ok=True)
                arr_name = 'any_results/formula_arr/' + main_id + '/' + n.formula + '.txt'
                df.to_pickle(arr_name)
    return 1

In [82]:
# Loop to download images from METASPACE for datasets
# Last arguement False == arrays, True = images
for prod_id, beta_id in cm3_wb_ds_prod_beta_dict.items():
    dl_img(prod_id, 
           beta_id, 
           'any_ds_db_msms_2020_Apr_28',
           0.5,
           False)

SMDataset(Technologie_Servier//TES-WBrat-vehicle (cloned from 2016-09-21_16h07m45s) | ID: 2020-04-29_00h28m08s)
SMDataset(whole body xenograft (1) [RMS norm] (cloned from 2017-05-17_19h49m04s) | ID: 2020-04-29_00h28m11s)
SMDataset(wb xenograft trp pathway dosed- rms_corrected (cloned from 2017-05-17_19h50m07s) | ID: 2020-04-29_00h28m14s)
SMDataset(servier_TT_mouse_wb_fmpts_derivatization_CHCA (cloned from 2017-05-29_07h28m52s) | ID: 2020-04-29_00h28m17s)
SMDataset(Servier_Ctrl_mouse_wb_median_plane_chca (cloned from 2017-08-03_15h09m06s) | ID: 2020-04-29_00h28m28s)
SMDataset(Servier_Ctrl_mouse_wb_lateral_plane_chca (cloned from 2017-08-03_15h09m51s) | ID: 2020-04-29_00h28m31s)
SMDataset(Servier_Ctrl_mouse_wb_lateral_plane_DHB (cloned from 2017-08-11_07h59m58s) | ID: 2020-04-29_00h28m35s)
SMDataset(Servier_Ctrl_mouse_wb_median_plane_DHB (cloned from 2017-08-11_08h01m02s) | ID: 2020-04-29_00h28m39s)


In [88]:
def copy_by_parent(new_id, report, img_true):
    ms_out = pd.read_csv(report)
    print(new_id)
    
    # iterate through ms_out, find img by formula, save in folder by id
    max_rows = ms_out.shape[0]
    counter = 0
    while counter < max_rows:
        ser = ms_out.iloc[counter,:]
        form = ser.formula
        par_id = ser.id_x
        name = ser.moleculeNames.join('_')
        ion_type = ser.par_frag
        
        if ion_type.find('p') != -1:
            out_ion = '_P'
        elif ion_type.find('f') != -1:
            out_ion = '_F'
        else:
            print('unknown ion type!')
        
        # Man edit line below for out path!
        outpath = 'any_results/by_id2_arr/' + new_id + '/' + par_id + '/'
        pathlib.Path(outpath).mkdir(parents=True, exist_ok=True)
        
        if img_true == True:
            infile = glob.glob('any_results/formula/' + 
                               new_id + '/' + form + '.png')[0]
            outfile = outpath + form + out_ion + '.png'

        else:
            infile = glob.glob('any_results/formula_arr/' + 
                               new_id + '/' + form + '.txt')[0]
            outfile = outpath + form + out_ion + '.txt'

        #print(infile, '\n', outfile)
        copyfile(infile, outfile)
        counter +=1
    print(new_id, ' Counter: ', counter)
    return 1

In [89]:
# Loop to group ion images or arrays by formula into by parent id

msms_reports = glob.glob('any_results/msms_theo_man_report/*.csv')
#msms_reports = glob.glob('any_results/msms_report/*.csv')
for report in msms_reports:
    new_id = report.split('/')[-1]
    new_id = new_id.split('_msms_')[0]
    copy_by_parent(new_id, report, False)

2016-09-21_16h07m45s
2016-09-21_16h07m45s  Counter:  53
2017-08-03_15h09m06s
2017-08-03_15h09m06s  Counter:  763
2017-05-17_19h49m04s
2017-05-17_19h49m04s  Counter:  2723
2017-05-17_19h50m07s
2017-05-17_19h50m07s  Counter:  3483
2017-08-11_07h59m58s
2017-08-11_07h59m58s  Counter:  724
2017-05-29_07h28m52s
2017-05-29_07h28m52s  Counter:  1829
2017-08-11_08h01m02s
2017-08-11_08h01m02s  Counter:  932
2017-08-03_15h09m51s
2017-08-03_15h09m51s  Counter:  565
